In [1]:
##加载相关包
import jieba
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import silhouette_score ,calinski_harabasz_score,davies_bouldin_score

d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [12]:
#导入数据
# import tensorflow_datasets as tfds
datas=pd.read_csv('D://作业//数据.csv')
datas
datas=list(datas['诉求内容'])
datas


['现市民要求部门撤销该罚款并给予合理解释请部门跟进处理谢谢',
 '现市民要求恢复原来的出入口并增设出入口请部门跟进处理谢谢',
 '请跟进处理谢谢',
 '轿车放在村路边被城管委托拖车公司拖车拖车前只有一个不明身份的打电话来说不能停放并未提及是什么部门以及拖车事宜被拖走后也没通知哪里找车我还是报警才找到车子的执法时不表明身份执法后也没有明确告知车子去向城管委托的拖车公司收费过高是否存在收费不合理我们连质疑的权利都没知情权也没车子就这样突然消失找到就要求缴费才能放行这事办的一点都不严谨也不合理要求整改以及退还拖车费用',
 '市民要求延后施工时间请跟进谢谢',
 '现市民要求学校可以让学生自由选择在校就餐和午休请跟进谢谢',
 '市民希望该驾校退款至少50%以上请跟进谢谢',
 '市民希望有关部门帮忙处理该情况请相关部门处理请跟进谢谢',
 '市民要求相关部门尽快处理请跟进谢谢',
 '现市民要求商家提供赠送永久性5张服务券的服务给予市民请协调谢谢',
 '东坑镇和昌.众筑中央松湖公寓6栋合同约定3月30日前交开发商冠成投资有限公司通知我5.1交楼结果5.1去到根本没有完工室内门未安装固定砖缝也未填灶具也未安装墙面更多缝及坑凹瑕疵…开发商也未出示装修工程验收合格的报告实质他们在工程未完成前提前叫业主收楼不想赔那么多的违约金',
 '市民要求相关部门及时整改噪音问题请跟进谢谢',
 '市民来电反映该医院购买了旁边银行的土地改造成住院间并将停车场改造成停尸间市民认为该医院将商业性质用地改造成医院使用不合理并且未经任何公示和征求周边居民意见便修建停尸间不合理市民希望了解是否合理 请跟进 谢谢',
 '现市民投诉太平派出所受理市民的案件请跟进谢谢',
 '请尽快跟进处理谢谢',
 '要求开发商退房或者换房或者拆掉现厕所重新改造请处理谢谢',
 '现市民要求相关部门尽快处理请跟进谢谢',
 '退款',
 '要求该业主拆除违建',
 '请处理违规停放',
 '我是2019年6月20日进入德尔智慧工厂科技有限公司的一名暖通工程师2022年6月29日德尔智慧工厂科技有限公司人事部经理刘同英女士约我谈话以2021年2月份的一个洁净室中央空调工程空调送风管采用PP板塑料风管因厚度薄变形后整改增加了费用为由把责任全部推在我个人身上对我进行劝退处理我不接受这个过错因为当时我出具的是1.2

In [22]:
#导入数据
data=pd.read_excel('D:/作业/2022镇街转派1清理.xlsx')
data=list(data['三级事项'])


In [3]:
# 去掉空字符
def delete_r_n(line):
    return line.replace('\r', '').replace('\n', '').strip()

# 加载停用词
def get_stop_words(stop_words_dir):
    stop_words = []

    with open(stop_words_dir, 'r', encoding='utf-8') as f_reader:
        for line in f_reader:
            line = delete_r_n(line)
            stop_words.append(line)

    stop_words = set(stop_words)
    return stop_words

# 加载停用词,stopwords为停用词列表
stopwords = get_stop_words(r'D:/作业/stop1.txt')



In [9]:
import jieba

# 加载自定义词典
#jieba.load_userdict("D:\wenben\8月12下午 文本分析\data\mysele_pro.txt")  # 替换为您自己的词典文件路径

# 进行分词
text = [' '.join(jieba.lcut(doc)) for doc in datas]
text

['现 市民 要求 部门 撤销 该 罚款 并 给予 合理 解释 请 部门 跟进 处理 谢谢',
 '现 市民 要求 恢复 原来 的 出入口 并 增设 出入口 请 部门 跟进 处理 谢谢',
 '请 跟进 处理 谢谢',
 '轿车 放在 村 路边 被 城管 委托 拖车 公司 拖车 拖车 前 只有 一个 不明 身份 的 打电话 来说 不能 停放 并未 提及 是 什么 部门 以及 拖车 事宜 被 拖走 后 也 没 通知 哪里找 车 我 还是 报警 才 找到 车子 的 执法 时 不 表明 身份 执法 后 也 没有 明确 告知 车子 去向 城管 委托 的 拖车 公司 收费 过高 是否 存在 收费 不合理 我们 连 质疑 的 权利 都 没 知情权 也 没 车子 就 这样 突然 消失 找到 就 要求 缴费 才能 放行 这事办 的 一点 都 不 严谨 也 不合理 要求 整改 以及 退还 拖车 费用',
 '市民 要求 延后 施工 时间 请 跟进 谢谢',
 '现 市民 要求 学校 可以 让 学生 自由选择 在校 就餐 和 午休 请 跟进 谢谢',
 '市民 希望 该 驾校 退款 至少 50% 以上 请 跟进 谢谢',
 '市民 希望 有关 部门 帮忙 处理 该 情况 请 相关 部门 处理 请 跟进 谢谢',
 '市民 要求 相关 部门 尽快 处理 请 跟进 谢谢',
 '现 市民 要求 商家 提供 赠送 永久性 5 张 服务 券 的 服务 给予 市民 请 协调 谢谢',
 '东坑镇 和昌 . 众筑 中央 松湖 公寓 6 栋 合同 约定 3 月 30 日前 交 开发商 冠成 投资 有限公司 通知 我 5.1 交楼 结果 5.1 去 到 根本 没有 完工 室内 门未 安装 固定 砖缝 也 未 填 灶具 也 未 安装 墙面 更多缝 及 坑 凹 瑕疵 … 开发商 也 未 出示 装修 工程验收 合格 的 报告 实质 他们 在 工程 未 完成 前提 前 叫 业主 收楼 不想 赔 那么 多 的 违约金',
 '市民 要求 相关 部门 及时 整改 噪音 问题 请 跟进 谢谢',
 '市民 来电 反映 该 医院 购买 了 旁边 银行 的 土地 改造 成 住院 间 并 将 停车场 改造 成 停尸间 市民 认为 该 医院 将 商业 性质 用地 改造 成 医院 使用 不合理 并且 未经 任何 公示 和 征

In [8]:
# 去除停用词
filtered_text = [' '.join([word for word in jieba.lcut(doc) if word not in stopwords]) for doc in datas]
# 假设你已经定义了 filtered_text 变量，它包含了经过分词和去除停用词后的文本

# 打印前几个处理后的文本
for doc in filtered_text[:5]:  # 这里选择打印前5个文本作为示例
    print(doc)


要求 部门 撤销 该 罚款 并 给予 合理 解释 请 部门 跟进 处理
要求 恢复 原来 出入口 并 增设 出入口 请 部门 跟进 处理
请 跟进 处理
轿车 放在 村 路边 被 城管 委托 拖车 公司 拖车 拖车 前 只有 不明 身份 打电话 来说 不能 停放 并未 提及 是 什么 部门 以及 拖车 事宜 被 拖走 后 也 没 通知 哪里找 车 我 还是 报警 才 找到 车子 执法 表明 身份 执法 后 也 没有 明确 告知 车子 去向 城管 委托 拖车 公司 收费 过高 是否 存在 收费 不合理 我们 连 质疑 权利 没 知情权 也 没 车子 就 这样 突然 消失 找到 就 要求 缴费 才能 放行 这事办 一点 严谨 也 不合理 要求 整改 以及 退还 拖车 费用
要求 延后 施工 请 跟进


In [10]:
for doc in filtered_text[:5]:  # 这里选择打印前5个文本作为示例
    print(doc)


要求 部门 撤销 该 罚款 并 给予 合理 解释 请 部门 跟进 处理
要求 恢复 原来 出入口 并 增设 出入口 请 部门 跟进 处理
请 跟进 处理
轿车 放在 村 路边 被 城管 委托 拖车 公司 拖车 拖车 前 只有 不明 身份 打电话 来说 不能 停放 并未 提及 是 什么 部门 以及 拖车 事宜 被 拖走 后 也 没 通知 哪里找 车 我 还是 报警 才 找到 车子 执法 表明 身份 执法 后 也 没有 明确 告知 车子 去向 城管 委托 拖车 公司 收费 过高 是否 存在 收费 不合理 我们 连 质疑 权利 没 知情权 也 没 车子 就 这样 突然 消失 找到 就 要求 缴费 才能 放行 这事办 一点 严谨 也 不合理 要求 整改 以及 退还 拖车 费用
要求 延后 施工 请 跟进


In [35]:
text = [' '.join(re.findall(r'[\u4e00-\u9fa5]+', doc.encode('utf-8').decode('utf-8')))
          for doc in datas]

In [36]:
text = [' '.join(jieba.lcut(doc))
          for doc in datas]


In [ ]:
datas

In [37]:
text

['诉求 内容']

In [9]:
wordcounter = CountVectorizer()
# 选择要保留的词汇

#reserved_words = [
    "了解", "做好", "公司", "分局", "前往", "加强", "协调", "单位", "及时", "双方",
    "发现", "告知", "周边", "咨询", "噪音", "城市", "城管", "处理", "存在", "安排",
    "居民", "巡查", "建议", "影响", "情况", "投诉", "接到", "提供", "收到", "教育",
    "整改", "服务", "查看", "核查", "核酸", "检测", "沟通", "热线", "现场", "疑问",
    "疫情", "监督", "目前", "相关", "社区", "立即", "管理", "组织", "经查", "继续",
    "综合", "群众", "落实", "行为", "表示", "要求", "规定", "解决", "诉求", "调查",
    "责令", "跟进", "路段", "车辆", "进一步", "进行", "通知", "造成", "防控", "需要"
]

from sklearn.feature_extraction.text import CountVectorizer


IndentationError: unexpected indent (<ipython-input-9-ffe9ff2ea1c7>, line 5)

In [26]:
wordcounter = CountVectorizer()
from sklearn.feature_extraction.text import CountVectorizer
# 计算TF-IDF
tfidf_matrix = wordcounter.fit_transform(text)

# 获取特征词汇列表
features = wordcounter.get_feature_names_out()


In [12]:
reserved_words1 = [
    "了解", "做好",  "前往", "加强", "协调",  "及时", "双方","咨询","安排", "教育","整改", "服务","沟通","立即","经查", "综合", "群众", "落实",  "解决", "诉求", "调查"
]



In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# 初始化CountVectorizer
count_vectorizer = CountVectorizer()

# 计算词频
count_matrix = count_vectorizer.fit_transform(text)

# 获取特征词汇列表
features = count_vectorizer.get_feature_names_out()

# 初始化一个存储词频总和的列表
word_frequencies = []

# 遍历TF-IDF矩阵的每个文本
for row in tfidf_matrix:
    # 将矩阵转换为数组并将其展平
    tfidf_values = row.toarray().flatten()
    
    # 初始化一个字典来映射特征词汇到它们在数组中的位置
    feature_index_map = {feature: index for index, feature in enumerate(features)}
    
    # 获取保留词汇的TF-IDF值，并将它们相加
    reserved_tfidf_sum = sum(tfidf_values[feature_index_map.get(word, -1)] for word in reserved_words1 if feature_index_map.get(word, -1) != -1)
    

In [17]:
reserved_words2 = [
    "了解", "做好",  "前往", "加强", "协调",  "及时", "双方","咨询","安排", "教育","整改", "服务","沟通","立即","经查", "综合", "群众", "落实",  "解决", "诉求", "调查"
]
# 遍历TF-IDF矩阵的每个文本
for row in tfidf_matrix:
    # 将矩阵转换为数组并将其展平
    tfidf_values = row.toarray().flatten()
    
    # 初始化一个字典来映射特征词汇到它们在数组中的位置
    feature_index_map = {feature: index for index, feature in enumerate(features)}
    
    # 获取保留词汇的TF-IDF值，并将它们相加
    reserved_tfidf_sum = sum(tfidf_values[feature_index_map.get(word, -1)] for word in reserved_words1 if feature_index_map.get(word, -1) != -1)
    
    # 将总和添加到列表中
    tfidf_sums.append(reserved_tfidf_sum)

       
    # 将总和添加到列表中
    reserved_tfidf_sum .append(reserved_tfidf_sum)

In [ ]:
    # 将总和添加到列表中
tfidf_sums.append(reserved_tfidf_sum)

In [25]:
import pandas as pd

# 将 tfidf_sums 转换为 DataFrame
tfidf_df = pd.DataFrame({'TF-IDF总和': tfidf_sums})

# 查看前几行
print(tfidf_df)
tfidf_df.to_excel('tfidf_summary.xlsx', index=False)


NameError: name 'tfidf_sums' is not defined

In [32]:
data2=list(data['诉求内容'])

TypeError: list indices must be integers or slices, not str

In [28]:
from snownlp import SnowNLP

# 初始化一个空的情感分数列表
sentiment_scores = []

# 遍历 TF-IDF 值列表并进行情感分析
for tfidf_value in tfidf_sums:
    s = SnowNLP(tfidf_value)
    sentiment_scores.append(s.sentiments)

# 将情感分数添加到数据中（如果需要）
# 例如，你可以将情感分数添加到与每个 TF-IDF 值相关联的数据行中

# 打印或保存情感分数
print(sentiment_scores)


TypeError: list indices must be integers or slices, not str

In [2]:
data=pd.read_excel('D:/作业/2022镇街转派1清理.xlsx')
data2=list(data['诉求内容'])